In [276]:
%load_ext autoreload
%autoreload 2

from mcts import * 
from tictactoe import *
from utils import *
import numpy as np
import random
import collections

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### run from initial state

In [306]:
board = np.array([[ 1 , 1 , 0 ], 
                  [-1 , 0 , 0 ], 
                  [ 0 , 0 , 0 ]]) 

state = GameState(board, player = -1)
root = MCTSNode(state)
agent = MCTSAgent(root)

next = agent.run_mcts(1000)
dot = visualize_mcts_tree(root)
dot.render('mcts_tree_visualization', format='png', view=True)


'mcts_tree_visualization.png'

### user v. mcts bot

In [305]:
board = np.zeros((3, 3))
user = 1
state = GameState(board, user)
while not state.is_terminal():
    # get user input 
    state.print_board()
    print('----------')
    action = input('Enter the action in the format i,j: ')
    i, j = action.split(',')
    i, j = int(i), int(j)

    if board[i,j] != 0:
        print('Invalid move')

    state = state.take_action((i,j))
    if state.is_terminal():
        break

    # get agent response
    node = MCTSNode(state)
    agent = MCTSAgent(node)
    node = agent.step(node, 10000, auto_response=False)
    state = node.state
state.print_board()
print(state.get_winner())


-  -  -

-  -  -

-  -  -
----------



X  O  -

-  -  -

-  -  -
----------

X  O  -

X  -  -

-  -  O
----------

X  O  X

X  O  -

-  -  O
----------

X  O  X

X  O  X

O  -  O
----------


ValueError: not enough values to unpack (expected 2, got 1)